In [32]:
import pandas as pd

In [345]:
from sshtunnel import SSHTunnelForwarder

AWS_IP_ADDRESS = '54.185.236.146'
AWS_USERNAME = 'luke'
SSH_KEY_PATH = '/users/luketibbott/.ssh/id_rsa'

server = SSHTunnelForwarder(
    AWS_IP_ADDRESS,
    ssh_username=AWS_USERNAME,
    ssh_pkey=SSH_KEY_PATH,
    remote_bind_address=('localhost', 5432),
)

server.start()
print(server.is_active, server.is_alive, server.local_bind_port)

True True 62972


In [346]:
from sqlalchemy import create_engine

# Postgres username, password, and database name
POSTGRES_IP_ADDRESS = 'localhost' ## This is localhost because SSH tunnel is active
POSTGRES_PORT = str(server.local_bind_port)
POSTGRES_USERNAME = 'luke'     ## CHANGE THIS TO YOUR POSTGRES USERNAME
POSTGRES_PASSWORD = 'luke' ## CHANGE THIS TO YOUR POSTGRES PASSWORD
POSTGRES_DBNAME = 'tennis'

# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME, 
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_IP_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

# Create the connection
cnx = create_engine(postgres_str)

In [347]:
cnx.table_names()

['aus_ladies_2013',
 'aus_men_2013',
 'us_men_2013',
 'us_ladies_2013',
 'french_men_2013',
 'french_ladies_2013',
 'wimbledon_men_2013',
 'wimbledon_ladies_2013']

**Question 1:** Using the same tennis data, find the number of matches played by each player in each tournament. (Remember that a player can be present as both player1 or player2).

In [168]:
pd.read_sql_query('''
                         /* Women player 1 */
                    WITH t1 AS (SELECT DISTINCT al.player1 ALp1, COUNT(al.player1) ALc1
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t2 AS (SELECT DISTINCT fl.player1 FLp1, COUNT(fl.player1) FLc1
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t3 AS (SELECT DISTINCT usl.player1 USLp1, COUNT(usl.player1) USLc1
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t4 AS (SELECT DISTINCT wl.player1 WLp1, COUNT(wl.player1) WLc1
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1),
                         /* Women player 2 */       
                         t5 AS (SELECT DISTINCT al.player2 ALp2, COUNT(al.player2) ALc2
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t6 AS (SELECT DISTINCT fl.player2 FLp2, COUNT(fl.player2) FLc2
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t7 AS (SELECT DISTINCT usl.player2 USLp2, COUNT(usl.player2) USLc2
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t8 AS (SELECT DISTINCT wl.player2 WLp2, COUNT(wl.player2) WLc2
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1), 
                         /* Men player 1 */       
                         t9 AS (SELECT DISTINCT am.player1 amp1, COUNT(am.player1) amc1
                                FROM aus_men_2013 am
                                GROUP BY 1),
                         t10 AS (SELECT DISTINCT fm.player1 fmp1, COUNT(fm.player1) fmc1
                                FROM french_men_2013 fm
                                GROUP BY 1),
                         t11 AS (SELECT DISTINCT usm.player1 usmp1, COUNT(usm.player1) usmc1
                                 FROM us_men_2013 usm
                                 GROUP BY 1),
                         t12 AS (SELECT DISTINCT wm.player1 wmp1, COUNT(wm.player1) wmc1
                                FROM wimbledon_men_2013 wm
                                GROUP BY 1),
                        /* Men player 2 */
                         t13 AS (SELECT DISTINCT am.player2 amp2, COUNT(am.player2) amc2
                                FROM aus_men_2013 am
                                GROUP BY 1),
                         t14 AS (SELECT DISTINCT fm.player2 fmp2, COUNT(fm.player2) fmc2
                                FROM french_men_2013 fm
                                GROUP BY 1),
                         t15 AS (SELECT DISTINCT usm.player2 usmp2, COUNT(usm.player2) usmc2
                                 FROM us_men_2013 usm
                                 GROUP BY 1),
                         t16 AS (SELECT DISTINCT wm.player2 wmp2, COUNT(wm.player2) wmc2
                                FROM wimbledon_men_2013 wm
                                GROUP BY 1),
                                
                         women_p1 AS (SELECT *
                                FROM t1
                                FULL OUTER JOIN t2
                                ON t1.ALp1 = t2.FLp1
                                FULL OUTER JOIN t3
                                ON t1.ALp1 = t3.USLp1 AND t2.FLp1 = t3.USLp1
                                FULL OUTER JOIN t4
                                ON t1.ALp1 = t4.WLp1 AND t2.FLp1 = t4.WLp1 AND t3.USLp1 = t4.WLp1),
                                
                         women_p2 AS (SELECT *
                                 FROM t5
                                 FULL OUTER JOIN t6
                                 ON t5.ALp2 = t6.FLp2
                                 FULL OUTER JOIN t7
                                 ON t5.ALp2 = t7.USLp2 AND t6.FLp2 = t7.USLp2
                                 FULL OUTER JOIN t8
                                 ON t5.ALp2 = t8.WLp2 AND t6.FLp2 = t8.WLp2 AND t7.USLp2 = t8.WLp2),
                                 
                         men_p1 AS (SELECT *
                              FROM t9
                              FULL OUTER JOIN t10
                              ON t9.amp1 = t10.fmp1
                              FULL OUTER JOIN t11
                              ON t9.amp1 = t11.usmp1 AND t10.fmp1 = t11.usmp1
                              FULL OUTER JOIN t12
                              ON t9.amp1 = t12.wmp1 AND t10.fmp1 = t12.wmp1 AND t11.usmp1 = t12.wmp1),
                              
                         men_p2 AS (SELECT *
                              FROM t13
                              FULL OUTER JOIN t14
                              ON t13.amp2 = t14.fmp2
                              FULL OUTER JOIN t15
                              ON t13.amp2 = t15.usmp2 AND t14.fmp2 = t15.usmp2
                              FULL OUTER JOIN t16
                              ON t13.amp2 = t16.wmp2 AND t14.fmp2 = t16.wmp2 AND t15.usmp2 = t16.wmp2),
                         
                         men_app AS (SELECT mp1.amp1, COALESCE((mp1.amc1 + mp2.amc2), 0) aus_men_app,
                                     mp1.fmp1, COALESCE((mp1.fmc1 + mp2.fmc2), 0) french_men_app,
                                     mp1.usmp1, COALESCE((mp1.usmc1 + mp2.usmc2), 0) us_men_app,
                                     mp1.wmp1, COALESCE((mp1.wmc1 + mp2.wmc2), 0) wimb_men_app
                                     FROM men_p1 mp1
                                     JOIN men_p2 mp2
                                     ON mp1.amp1 = mp2.amp2 OR mp1.fmp1 = mp2.fmp2
                                        OR mp1.usmp1 = mp2.usmp2 OR mp1.wmp1 = mp2.wmp2),
                                        
                         unique_men AS (SELECT RIGHT(amp1, CHAR_LENGTH(amp1) - POSITION (' ' IN amp1))
                                      AS men_players
                                      FROM men_app
                                      WHERE amp1 IS NOT NULL
                                      UNION 
                                      SELECT RIGHT(fmp1, CHAR_LENGTH(fmp1) - POSITION (' ' IN fmp1))
                                      FROM men_app
                                      WHERE fmp1 IS NOT NULL
                                      UNION
                                      SELECT RIGHT(usmp1, CHAR_LENGTH(usmp1) - POSITION(' ' IN usmp1))
                                      FROM men_app
                                      WHERE usmp1 IS NOT NULL
                                      UNION
                                      SELECT RIGHT(wmp1, CHAR_LENGTH(wmp1) - POSITION('.' IN wmp1))
                                      FROM men_app
                                      WHERE wmp1 IS NOT NULL),
                                 
                         women_app AS (SELECT wp1.alp1, COALESCE((wp1.alc1 + wp2.alc2), 0) aus_ladies_app, 
                                    wp1.flp1, COALESCE((wp1.flc1 + wp2.flc2), 0) french_ladies_app,
                                    wp1.uslp1, COALESCE((wp1.uslc1 + wp2.uslc2), 0) us_ladies_app,
                                    wp1.wlp1, COALESCE((wp1.wlc1 + wp2.wlc2), 0) wimb_ladies_app
                                    FROM women_p1 wp1
                                    JOIN women_p2 wp2
                                    ON wp1.alp1 = wp2.alp2 OR wp1.flp1 = wp2.flp2
                                       OR wp1.uslp1 = wp2.uslp2 OR wp1.wlp1 = wp2.wlp2),
                                       
                         unique_women AS (SELECT RIGHT(alp1, CHAR_LENGTH(alp1) - POSITION(' ' IN alp1))
                                    AS women_players
                                    FROM women_app
                                    WHERE alp1 IS NOT NULL
                                    UNION
                                    SELECT RIGHT(flp1, CHAR_LENGTH(flp1) - POSITION(' ' IN flp1))
                                    FROM women_app
                                    WHERE flp1 IS NOT NULL
                                    UNION 
                                    SELECT RIGHT(uslp1, CHAR_LENGTH(uslp1) - POSITION(' ' IN uslp1))
                                    FROM women_app
                                    WHERE uslp1 IS NOT NULL
                                    UNION
                                    SELECT RIGHT(wlp1, CHAR_LENGTH(wlp1) - POSITION('.' IN wlp1))
                                    FROM women_app
                                    WHERE wlp1 IS NOT NULL),
                                    
                    tourn_appearances_men AS (SELECT um.men_players, ma.aus_men_app australian_appearances,
                                        ma.french_men_app french_appearances, ma.us_men_app US_appearances,
                                        ma.wimb_men_app wimbledon_appearances
                                    FROM unique_men um
                                    JOIN men_app ma
                                    ON RIGHT(amp1, CHAR_LENGTH(amp1) - POSITION (' ' IN amp1)) = um.men_players
                                        OR RIGHT(fmp1, CHAR_LENGTH(fmp1) - POSITION (' ' IN fmp1)) = um.men_players
                                        OR RIGHT(usmp1, CHAR_LENGTH(usmp1) - POSITION(' ' IN usmp1)) = um.men_players
                                        OR RIGHT(wmp1, CHAR_LENGTH(wmp1) - POSITION('.' IN wmp1)) = um.men_players),
                                    
                    tourn_appearances_women AS (SELECT uw.women_players, wa.aus_ladies_app australian_appearances, 
                                       wa.french_ladies_app french_appearances, wa.us_ladies_app US_appearances,
                                       wa.wimb_ladies_app wimbledon_appearances
                                    FROM unique_women uw
                                    JOIN women_app wa
                                    ON RIGHT(wa.alp1, CHAR_LENGTH(alp1) - POSITION(' ' IN alp1)) = uw.women_players
                                        OR RIGHT(wa.flp1, CHAR_LENGTH(flp1) - POSITION(' ' IN flp1)) = uw.women_players
                                        OR RIGHT(wa.uslp1, CHAR_LENGTH(uslp1) - POSITION(' ' IN uslp1)) = uw.women_players
                                        OR RIGHT(wa.wlp1, CHAR_LENGTH(wlp1) - POSITION('.' IN wlp1)) = uw.women_players),
                                        
                    women_solution AS (SELECT women_players AS player, SUM(australian_appearances) aus_appear,
                                   SUM(french_appearances) fr_appear, SUM(us_appearances) us_appear,
                                   SUM(wimbledon_appearances) wimb_appear
                                    FROM tourn_appearances_women
                                    GROUP BY 1),
                                    
                    men_solution AS (SELECT men_players AS player, SUM(australian_appearances) aus_appear,
                                    SUM(french_appearances) fr_appear, SUM(us_appearances) us_appear,
                                    SUM(wimbledon_appearances) wimb_appear
                                    FROM tourn_appearances_men
                                    GROUP BY 1)
                    
        
                    SELECT *
                    FROM women_solution
                    UNION
                    SELECT *
                    FROM men_solution;
                  ''', cnx)

,player,aus_appear,fr_appear,us_appear,wimb_appear
0,Azarenka,0.0,6.0,0.0,0.0
1,Seppi,0.0,0.0,0.0,3.0
2,Haas,0.0,5.0,0.0,0.0
3,Larsson,0.0,2.0,0.0,0.0
4,Struff,0.0,0.0,0.0,2.0
5,Mattek-Sands,0.0,4.0,0.0,0.0
6,Tipsarevic,0.0,0.0,4.0,0.0
7,Kirilenko,0.0,5.0,0.0,0.0
8,Ebden,2.0,0.0,0.0,0.0
9,Bouchard,6.0,0.0,2.0,3.0


Above are the players and number of appearances in each tournament.

**Question 2:** Who has played the most matches total in all of US Open, AUST Open, French Open? Answer this both for men and women.

I'll use my last query to answer this.

In [132]:
pd.read_sql_query('''
                         /* Women player 1 */
                    WITH t1 AS (SELECT DISTINCT al.player1 ALp1, COUNT(al.player1) ALc1
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t2 AS (SELECT DISTINCT fl.player1 FLp1, COUNT(fl.player1) FLc1
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t3 AS (SELECT DISTINCT usl.player1 USLp1, COUNT(usl.player1) USLc1
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t4 AS (SELECT DISTINCT wl.player1 WLp1, COUNT(wl.player1) WLc1
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1),
                         /* Women player 2 */       
                         t5 AS (SELECT DISTINCT al.player2 ALp2, COUNT(al.player2) ALc2
                                FROM aus_ladies_2013 al
                                GROUP BY 1),
                         t6 AS (SELECT DISTINCT fl.player2 FLp2, COUNT(fl.player2) FLc2
                                FROM french_ladies_2013 fl
                                GROUP BY 1),
                         t7 AS (SELECT DISTINCT usl.player2 USLp2, COUNT(usl.player2) USLc2
                                FROM us_ladies_2013 usl
                                GROUP BY 1),
                         t8 AS (SELECT DISTINCT wl.player2 WLp2, COUNT(wl.player2) WLc2
                                FROM wimbledon_ladies_2013 wl
                                GROUP BY 1), 
                         /* Men player 1 */       
                         t9 AS (SELECT DISTINCT am.player1 amp1, COUNT(am.player1) amc1
                                FROM aus_men_2013 am
                                GROUP BY 1),
                         t10 AS (SELECT DISTINCT fm.player1 fmp1, COUNT(fm.player1) fmc1
                                FROM french_men_2013 fm
                                GROUP BY 1),
                         t11 AS (SELECT DISTINCT usm.player1 usmp1, COUNT(usm.player1) usmc1
                                 FROM us_men_2013 usm
                                 GROUP BY 1),
                         t12 AS (SELECT DISTINCT wm.player1 wmp1, COUNT(wm.player1) wmc1
                                FROM wimbledon_men_2013 wm
                                GROUP BY 1),
                        /* Men player 2 */
                         t13 AS (SELECT DISTINCT am.player2 amp2, COUNT(am.player2) amc2
                                FROM aus_men_2013 am
                                GROUP BY 1),
                         t14 AS (SELECT DISTINCT fm.player2 fmp2, COUNT(fm.player2) fmc2
                                FROM french_men_2013 fm
                                GROUP BY 1),
                         t15 AS (SELECT DISTINCT usm.player2 usmp2, COUNT(usm.player2) usmc2
                                 FROM us_men_2013 usm
                                 GROUP BY 1),
                         t16 AS (SELECT DISTINCT wm.player2 wmp2, COUNT(wm.player2) wmc2
                                FROM wimbledon_men_2013 wm
                                GROUP BY 1),
                                
                         women_p1 AS (SELECT *
                                FROM t1
                                FULL OUTER JOIN t2
                                ON t1.ALp1 = t2.FLp1
                                FULL OUTER JOIN t3
                                ON t1.ALp1 = t3.USLp1 AND t2.FLp1 = t3.USLp1
                                FULL OUTER JOIN t4
                                ON t1.ALp1 = t4.WLp1 AND t2.FLp1 = t4.WLp1 AND t3.USLp1 = t4.WLp1),
                                
                         women_p2 AS (SELECT *
                                 FROM t5
                                 FULL OUTER JOIN t6
                                 ON t5.ALp2 = t6.FLp2
                                 FULL OUTER JOIN t7
                                 ON t5.ALp2 = t7.USLp2 AND t6.FLp2 = t7.USLp2
                                 FULL OUTER JOIN t8
                                 ON t5.ALp2 = t8.WLp2 AND t6.FLp2 = t8.WLp2 AND t7.USLp2 = t8.WLp2),
                                 
                         men_p1 AS (SELECT *
                              FROM t9
                              FULL OUTER JOIN t10
                              ON t9.amp1 = t10.fmp1
                              FULL OUTER JOIN t11
                              ON t9.amp1 = t11.usmp1 AND t10.fmp1 = t11.usmp1
                              FULL OUTER JOIN t12
                              ON t9.amp1 = t12.wmp1 AND t10.fmp1 = t12.wmp1 AND t11.usmp1 = t12.wmp1),
                              
                         men_p2 AS (SELECT *
                              FROM t13
                              FULL OUTER JOIN t14
                              ON t13.amp2 = t14.fmp2
                              FULL OUTER JOIN t15
                              ON t13.amp2 = t15.usmp2 AND t14.fmp2 = t15.usmp2
                              FULL OUTER JOIN t16
                              ON t13.amp2 = t16.wmp2 AND t14.fmp2 = t16.wmp2 AND t15.usmp2 = t16.wmp2),
                         
                         men_app AS (SELECT mp1.amp1, COALESCE((mp1.amc1 + mp2.amc2), 0) aus_men_app,
                                     mp1.fmp1, COALESCE((mp1.fmc1 + mp2.fmc2), 0) french_men_app,
                                     mp1.usmp1, COALESCE((mp1.usmc1 + mp2.usmc2), 0) us_men_app,
                                     mp1.wmp1, COALESCE((mp1.wmc1 + mp2.wmc2), 0) wimb_men_app
                                     FROM men_p1 mp1
                                     JOIN men_p2 mp2
                                     ON mp1.amp1 = mp2.amp2 OR mp1.fmp1 = mp2.fmp2
                                        OR mp1.usmp1 = mp2.usmp2 OR mp1.wmp1 = mp2.wmp2),
                                        
                         unique_men AS (SELECT amp1 AS full_name, RIGHT(amp1, CHAR_LENGTH(amp1) - POSITION (' ' IN amp1))
                                      AS men_players
                                      FROM men_app
                                      WHERE amp1 IS NOT NULL
                                      UNION 
                                      SELECT fmp1, RIGHT(fmp1, CHAR_LENGTH(fmp1) - POSITION (' ' IN fmp1))
                                      FROM men_app
                                      WHERE fmp1 IS NOT NULL
                                      UNION
                                      SELECT usmp1, RIGHT(usmp1, CHAR_LENGTH(usmp1) - POSITION(' ' IN usmp1))
                                      FROM men_app
                                      WHERE usmp1 IS NOT NULL
                                      UNION
                                      SELECT wmp1, RIGHT(wmp1, CHAR_LENGTH(wmp1) - POSITION('.' IN wmp1))
                                      FROM men_app
                                      WHERE wmp1 IS NOT NULL),
                                 
                         women_app AS (SELECT wp1.alp1, COALESCE((wp1.alc1 + wp2.alc2), 0) aus_ladies_app, 
                                    wp1.flp1, COALESCE((wp1.flc1 + wp2.flc2), 0) french_ladies_app,
                                    wp1.uslp1, COALESCE((wp1.uslc1 + wp2.uslc2), 0) us_ladies_app,
                                    wp1.wlp1, COALESCE((wp1.wlc1 + wp2.wlc2), 0) wimb_ladies_app
                                    FROM women_p1 wp1
                                    JOIN women_p2 wp2
                                    ON wp1.alp1 = wp2.alp2 OR wp1.flp1 = wp2.flp2
                                       OR wp1.uslp1 = wp2.uslp2 OR wp1.wlp1 = wp2.wlp2),
                                       
                         unique_women AS (SELECT alp1 AS full_name, RIGHT(alp1, CHAR_LENGTH(alp1) - POSITION(' ' IN alp1))
                                    AS women_players
                                    FROM women_app
                                    WHERE alp1 IS NOT NULL
                                    UNION
                                    SELECT flp1, RIGHT(flp1, CHAR_LENGTH(flp1) - POSITION(' ' IN flp1))
                                    FROM women_app
                                    WHERE flp1 IS NOT NULL
                                    UNION 
                                    SELECT uslp1, RIGHT(uslp1, CHAR_LENGTH(uslp1) - POSITION(' ' IN uslp1))
                                    FROM women_app
                                    WHERE uslp1 IS NOT NULL
                                    UNION
                                    SELECT wlp1, RIGHT(wlp1, CHAR_LENGTH(wlp1) - POSITION('.' IN wlp1))
                                    FROM women_app
                                    WHERE wlp1 IS NOT NULL),
                                    
                    tourn_appearances_men AS (SELECT um.full_name, ma.aus_men_app australian_appearances,
                                        ma.french_men_app french_appearances, ma.us_men_app US_appearances,
                                        ma.wimb_men_app wimbledon_appearances
                                    FROM unique_men um
                                    JOIN men_app ma
                                    ON RIGHT(amp1, CHAR_LENGTH(amp1) - POSITION (' ' IN amp1)) = um.men_players
                                        OR RIGHT(fmp1, CHAR_LENGTH(fmp1) - POSITION (' ' IN fmp1)) = um.men_players
                                        OR RIGHT(usmp1, CHAR_LENGTH(usmp1) - POSITION(' ' IN usmp1)) = um.men_players
                                        OR RIGHT(wmp1, CHAR_LENGTH(wmp1) - POSITION('.' IN wmp1)) = um.men_players),
                                    
                    tourn_appearances_women AS (SELECT uw.full_name, wa.aus_ladies_app australian_appearances, 
                                       wa.french_ladies_app french_appearances, wa.us_ladies_app US_appearances,
                                       wa.wimb_ladies_app wimbledon_appearances
                                    FROM unique_women uw
                                    JOIN women_app wa
                                    ON RIGHT(wa.alp1, CHAR_LENGTH(alp1) - POSITION(' ' IN alp1)) = uw.women_players
                                        OR RIGHT(wa.flp1, CHAR_LENGTH(flp1) - POSITION(' ' IN flp1)) = uw.women_players
                                        OR RIGHT(wa.uslp1, CHAR_LENGTH(uslp1) - POSITION(' ' IN uslp1)) = uw.women_players
                                        OR RIGHT(wa.wlp1, CHAR_LENGTH(wlp1) - POSITION('.' IN wlp1)) = uw.women_players),
                                        
                    women_solution AS (SELECT full_name AS player, SUM(australian_appearances) aus_appear,
                                   SUM(french_appearances) fr_appear, SUM(us_appearances) us_appear,
                                   SUM(wimbledon_appearances) wimb_appear
                                    FROM tourn_appearances_women
                                    GROUP BY 1),
                                    
                    men_solution AS (SELECT full_name AS player, SUM(australian_appearances) aus_appear,
                                    SUM(french_appearances) fr_appear, SUM(us_appearances) us_appear,
                                    SUM(wimbledon_appearances) wimb_appear
                                    FROM tourn_appearances_men
                                    GROUP BY 1)
                    
        
                    SELECT player, (aus_appear + fr_appear + us_appear + wimb_appear) AS total_appearances
                    FROM women_solution
                    ORDER BY 2 DESC
                    LIMIT 1;  
                  ''', cnx)

,player,total_appearances
0,A.Radwanska,14.0


A. Radwanska had the most appearances over the entire tournament.

**Question 3:** Who has the highest first serve percentage? (Just the maximum value in a single match.)

In [146]:
cnx.table_names()

['aus_ladies_2013',
 'aus_men_2013',
 'us_men_2013',
 'us_ladies_2013',
 'french_men_2013',
 'french_ladies_2013',
 'wimbledon_men_2013',
 'wimbledon_ladies_2013']

In [155]:
pd.read_sql_query(''' 
                    WITH t1 AS (SELECT player1, ssp_1
                    FROM aus_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t2 AS (SELECT player2, ssp_2
                    FROM aus_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t3 AS (SELECT player1, ssp_1
                    FROM aus_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t4 AS (SELECT player2, ssp_2
                    FROM aus_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t5 AS (SELECT player1, ssp_1
                    FROM french_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t6 AS (SELECT player2, ssp_2
                    FROM french_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t7 AS (SELECT player1, ssp_1
                    FROM french_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t8 AS (SELECT player2, ssp_2
                    FROM french_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t9 AS (SELECT player1, ssp_1
                    FROM us_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t10 AS (SELECT player2, ssp_2
                    FROM us_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t11 AS (SELECT player1, ssp_1
                    FROM us_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t12 AS (SELECT player2, ssp_2
                    FROM us_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t13 AS (SELECT player1, ssp_1
                    FROM wimbledon_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t14 AS (SELECT player2, ssp_2
                    FROM wimbledon_ladies_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t15 AS (SELECT player1, ssp_1
                    FROM wimbledon_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    t16 AS (SELECT player2, ssp_2
                    FROM wimbledon_men_2013
                    ORDER BY 2 DESC
                    LIMIT 1),
                    
                    top_servers AS (SELECT *
                  FROM t1
                  UNION
                  SELECT *
                  FROM t2
                  UNION
                  SELECT *
                  FROM t3
                  UNION
                  SELECT *
                  FROM t4
                  UNION
                  SELECT *
                  FROM t5
                  UNION
                  SELECT *
                  FROM t6
                  UNION
                  SELECT *
                  FROM t7
                  UNION
                  SELECT *
                  FROM t8
                  UNION
                  SELECT *
                  FROM t9
                  UNION
                  SELECT *
                  FROM t10
                  UNION
                  SELECT *
                  FROM t11
                  UNION
                  SELECT *
                  FROM t12
                  UNION
                  SELECT *
                  FROM t13
                  UNION
                  SELECT *
                  FROM t14
                  UNION
                  SELECT *
                  FROM t15
                  UNION
                  SELECT *
                  FROM t16)
                  
                  SELECT player1, ssp_1
                  FROM top_servers
                  ORDER BY ssp_1 DESC
                  LIMIT 2;
                  ''', cnx)

,player1,ssp_1
0,Polona Hercog,100.0
1,Lukasz Kubot,62.0


Polona Hercog had a 100% serve rate. The next highest was Lukasz Kubot, at 62%.

**Question 4:** What are the unforced error percentages of the top three players with the most wins? (Unforced error percentage is % of points lost due to unforced errors. In a match, you have fields for number of points won by each player, and number of unforced errors for each field.)

Hint: SUM(double_faults) sums the contents of an entire column. For each row, to add the field values from two columns, the syntax SELECT name, double_faults + unforced_errors can be used.

Special bonus hint: To be careful about handling possible ties, consider using rank functions.

dbf_1, dbf_2

0 means player 2 wins, 1 means player 1 wins

tpw_1 - total points won by player 1
tpw_2 - total points won by player 2

In [298]:
pd.set_option('display.max_columns', 100)

In [299]:
pd.read_sql_query('''
                    SELECT *
                    FROM aus_ladies_2013;
                  ''', cnx)

,player1,player2,round,result,fnl1,fnl2,fsp_1,fsw_1,ssp_1,ssw_1,ace_1,dbf_1,wnr_1,ufe_1,bpc_1,bpw_1,npa_1,npw_1,tpw_1,st1_1,st2_1,st3_1,st4_1,st5_1,fsp_2,fsw_2,ssp_2,ssw_2,ace_2,dbf_2,wnr_2,ufe_2,bpc_2,bpw_2,npa_2,npw_2,tpw_2,st1_2,st2_2,st3_2,st4_2,st5_2
0,Serena Williams,Ashleigh Barty,1,1,2.0,0.0,59.0,20.0,41.0,8.0,6.0,2.0,31,17,5.0,10.0,10.0,11.0,58,6.0,6.0,NaN,None,None,51.0,14.0,49.0,11.0,1.0,4.0,5,18,0.0,0.0,2.0,4.0,31,2.0,1.0,NaN,None,None
1,Vesna Dolonc,Lara Arruabarrena,1,1,2.0,1.0,65.0,33.0,35.0,10.0,7.0,4.0,7,4,5.0,10.0,NaN,NaN,78,2.0,6.0,6.0,None,None,62.0,33.0,38.0,10.0,0.0,3.0,0,3,4.0,7.0,NaN,NaN,74,6.0,2.0,4.0,None,None
2,Pauline Parmentier,Karolina Pliskova,1,0,0.0,2.0,63.0,16.0,37.0,4.0,3.0,6.0,3,6,0.0,2.0,NaN,NaN,37,0.0,1.0,NaN,None,None,60.0,26.0,40.0,7.0,5.0,0.0,5,0,5.0,14.0,NaN,NaN,64,6.0,6.0,NaN,None,None
3,Heather Watson,Daniela Hantuchova,1,0,1.0,2.0,61.0,41.0,39.0,19.0,8.0,3.0,27,45,4.0,7.0,10.0,13.0,88,5.0,6.0,3.0,None,None,68.0,43.0,32.0,10.0,6.0,2.0,37,44,5.0,13.0,5.0,8.0,102,7.0,3.0,6.0,None,None
4,Samantha Stosur,Klara Zakopalova,1,1,2.0,0.0,65.0,28.0,35.0,11.0,6.0,1.0,19,18,7.0,10.0,7.0,10.0,74,6.0,6.0,NaN,None,None,50.0,18.0,50.0,7.0,1.0,5.0,27,27,4.0,14.0,10.0,15.0,60,3.0,4.0,NaN,None,None
5,Tsvetana Pironkova,Silvia Soler-Espinosa,1,1,2.0,0.0,62.0,28.0,38.0,12.0,5.0,0.0,30,21,3.0,5.0,4.0,7.0,68,6.0,6.0,NaN,None,None,50.0,18.0,50.0,14.0,3.0,2.0,11,23,0.0,3.0,6.0,6.0,44,3.0,2.0,NaN,None,None
6,Annika Beck,Petra Martic,1,1,2.0,0.0,67.0,18.0,33.0,8.0,0.0,0.0,8,10,6.0,11.0,3.0,3.0,52,6.0,6.0,NaN,None,None,50.0,8.0,50.0,4.0,2.0,6.0,7,34,0.0,2.0,3.0,7.0,25,0.0,0.0,NaN,None,None
7,Kiki Bertens,Ana Ivanovic,1,0,0.0,2.0,61.0,15.0,39.0,11.0,2.0,4.0,23,35,6.0,11.0,10.0,16.0,61,4.0,4.0,NaN,None,None,60.0,24.0,40.0,9.0,3.0,4.0,14,22,8.0,9.0,4.0,6.0,73,6.0,6.0,NaN,None,None
8,Roberta Vinci,Jie Zheng,1,0,0.0,2.0,66.0,23.0,34.0,7.0,3.0,3.0,20,28,3.0,10.0,7.0,12.0,56,4.0,3.0,NaN,None,None,68.0,26.0,32.0,10.0,0.0,2.0,26,25,5.0,7.0,1.0,2.0,67,6.0,6.0,NaN,None,None
9,Patricia Mayr-Achleitner,Madison Keys,1,0,1.0,2.0,71.0,48.0,29.0,14.0,0.0,1.0,0,1,5.0,12.0,NaN,NaN,113,2.0,7.0,7.0,None,None,52.0,45.0,48.0,30.0,11.0,4.0,11,4,8.0,10.0,NaN,NaN,135,6.0,6.0,9.0,None,None


In [369]:
pd.read_sql_query('''

                WITH t1 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM aus_ladies_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t2 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM aus_ladies_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    aus_ladies_wins AS (SELECT COALESCE(t1.player, t2.player), COALESCE(t1.wins_as_p2, 0) + COALESCE(t2.wins_as_p1, 0) tot_wins
                    FROM t1
                    FULL OUTER JOIN t2
                    ON t1.player = t2.player
                    ORDER BY 2 DESC),
                    
                    t3 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM aus_men_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t4 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM aus_men_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    aus_men_wins AS (SELECT COALESCE(t3.player, t4.player),
                        COALESCE(t3.wins_as_p2, 0) + COALESCE(t4.wins_as_p1, 0) tot_wins
                    FROM t3
                    FULL OUTER JOIN t4
                    ON t3.player = t4.player
                    ORDER BY 2 DESC),
                    
                    t5 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM french_ladies_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t6 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM french_ladies_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    french_ladies_wins AS (SELECT COALESCE(t5.player, t6.player),
                        COALESCE(t5.wins_as_p2, 0) + COALESCE(t6.wins_as_p1, 0) tot_wins
                    FROM t5
                    FULL OUTER JOIN t6
                    ON t5.player = t6.player
                    ORDER BY 2 DESC),
                    
                    t7 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM french_men_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t8 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM french_men_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    french_men_wins AS (SELECT COALESCE(t7.player, t8.player),
                        COALESCE(t7.wins_as_p2, 0) + COALESCE(t8.wins_as_p1, 0) tot_wins
                    FROM t7
                    FULL OUTER JOIN t8
                    ON t7.player = t8.player
                    ORDER BY 2 DESC),
                    
                    t9 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM us_ladies_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t10 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM us_ladies_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    us_ladies_wins AS (SELECT COALESCE(t9.player, t10.player),
                    COALESCE(t9.wins_as_p2, 0) + COALESCE(t10.wins_as_p1, 0) tot_wins
                    FROM t9
                    FULL OUTER JOIN t10
                    ON t9.player = t10.player
                    ORDER BY 2 DESC),
                    
                    t11 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM us_men_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t12 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM us_men_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    us_men_wins AS (SELECT COALESCE(t11.player, t12.player), 
                        COALESCE(t11.wins_as_p2, 0) + COALESCE(t12.wins_as_p1, 0) tot_wins
                    FROM t11
                    FULL OUTER JOIN t12
                    ON t11.player = t12.player
                    ORDER BY 2 DESC),
                    
                    t13 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM wimbledon_ladies_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t14 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM wimbledon_ladies_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    wimb_ladies_wins AS (SELECT COALESCE(t13.player, t14.player),
                        COALESCE(t13.wins_as_p2, 0) + COALESCE(t14.wins_as_p1, 0) tot_wins
                    FROM t13
                    FULL OUTER JOIN t14
                    ON t13.player = t14.player
                    ORDER BY 2 DESC),
                    
                    t15 AS(
                    SELECT player2 player, COUNT(player2) wins_as_p2
                    FROM wimbledon_men_2013 
                    WHERE result = 0 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    t16 AS (SELECT player1 player, COUNT(player1) wins_as_p1
                    FROM wimbledon_men_2013 
                    WHERE result = 1 AND fnl1 != fnl2 
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    wimb_men_wins AS (SELECT COALESCE(t15.player, t16.player), COALESCE(t15.wins_as_p2, 0) + COALESCE(t16.wins_as_p1, 0) tot_wins
                    FROM t15
                    FULL OUTER JOIN t16
                    ON t15.player = t16.player
                    ORDER BY 2 DESC),
                                        
                    lady_wins AS (SELECT DISTINCT al.coalesce,
                        RIGHT(al.coalesce, CHAR_LENGTH(al.coalesce) - POSITION(' ' IN al.coalesce)), al.tot_wins
                    FROM aus_ladies_wins al
                    UNION
                    SELECT DISTINCT fl.coalesce,
                        RIGHT(fl.coalesce, CHAR_LENGTH(fl.coalesce) - POSITION(' ' IN fl.coalesce)), fl.tot_wins
                    FROM french_ladies_wins fl
                    UNION
                    SELECT DISTINCT ul.coalesce,
                        RIGHT(ul.coalesce, CHAR_LENGTH(ul.coalesce) - POSITION(' ' IN ul.coalesce)), ul.tot_wins
                    FROM us_ladies_wins ul
                    UNION
                    SELECT DISTINCT wl.coalesce,
                        RIGHT(wl.coalesce, CHAR_LENGTH(wl.coalesce) - POSITION('.' IN wl.coalesce)), wl.tot_wins
                    FROM wimb_ladies_wins wl),
                    
                    men_wins AS (SELECT am.coalesce,
                    RIGHT(am.coalesce, CHAR_LENGTH(am.coalesce) - POSITION(' ' IN am.coalesce)), am.tot_wins
                    FROM aus_men_wins am
                    UNION
                    SELECT fm.coalesce, 
                        RIGHT(fm.coalesce, CHAR_LENGTH(fm.coalesce) - POSITION(' ' IN fm.coalesce)), fm.tot_wins
                    FROM french_men_wins fm
                    UNION
                    SELECT um.coalesce,
                        RIGHT(um.coalesce, CHAR_LENGTH(um.coalesce) - POSITION(' ' IN um.coalesce)), um.tot_wins
                    FROM us_men_wins um
                    UNION
                    SELECT wm.coalesce,
                        RIGHT(wm.coalesce, CHAR_LENGTH(wm.coalesce) - POSITION('.' IN wm.coalesce)), wm.tot_wins
                    FROM wimb_men_wins wm),
                    
                    top_women AS (SELECT lady_wins.coalesce AS player, SUM(lady_wins.tot_wins) AS tot_wins
                    FROM lady_wins
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    top_men AS (SELECT men_wins.coalesce AS player, SUM(men_wins.tot_wins) AS tot_wins
                    FROM men_wins
                    GROUP BY 1
                    ORDER BY 2 DESC),
                    
                    top_all AS (SELECT *
                    FROM top_men
                    UNION
                    SELECT *
                    FROM top_women),
                    
                    top_3 AS (SELECT ta.player, ta.tot_wins
                    FROM top_all ta
                    ORDER BY 2 DESC
                    LIMIT 3),
                    
                    stats AS (SELECT player1, player2, ufe_1, ufe_2, tpw_1, tpw_2
                        FROM aus_men_2013
                        UNION
                        SELECT player1, player2, ufe_1, ufe_2, tpw_1, tpw_2
                        FROM french_men_2013),
                        
                    /*SELECT *
                    FROM stats s
                    JOIN top_3 
                    ON 
                    RIGHT(s.player1, CHAR_LENGTH(s.player1) - POSITION(' ' IN s.player1)) = top_3.player
                    OR RIGHT(s.player2, CHAR_LENGTH(s.player2) - POSITION(' ' IN s.player2)) = top_3.player; */
                    
                    unforced_err_as_p1 AS (
                        SELECT s.player1, (CAST(s.ufe_1 AS DECIMAL) / CAST(s.tpw_1 AS DECIMAL)) ufep
                        FROM top_3
                        JOIN stats s
                        ON s.player1 = top_3.player),
                        
                    unforced_err_as_p2 AS (
                        SELECT top_3.player, (CAST(s.ufe_2 AS DECIMAL) / CAST(s.tpw_2 AS DECIMAL)) ufep
                        FROM top_3
                        JOIN stats s
                        ON s.player2 = top_3.player
                        ),
                        
                    unforced_err_all AS (SELECT *
                    FROM unforced_err_as_p1
                    UNION
                    SELECT *
                    FROM unforced_err_as_p2)
                        
                    SELECT player1, AVG(ufep)*100 AS unforced_err_percentage
                    FROM unforced_err_all
                    GROUP BY 1;
                  ''', cnx)

,player1,unforced_err_percentage
0,Stanislas Wawrinka,34.221738
1,Rafael Nadal,24.305803
2,Roger Federer,31.177935


The top three players by wins are all men. 

Their unforced error percentages are:

|Player|Unforced error percentage|
|-------|:------------------------:|
|Stanislas Wawrinka| 34.22 |
|Rafael Nadal | 23.31 |
|Roger Federer | 31.18 |